In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from owi_data_2_pandas.io import API
import datetime
from pytz import utc
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import matplotlib.dates as mdates

# sklearn imports
from sklearn.model_selection import train_test_split

# oma_tracking imports
from oma_tracking.data.preprocessing import sin_cos_angle_inputs

# Import vub Meetenet Vlaamse Banken API functions
from vubmvbc.config import Credentials
from vubmvbc.client import Base
from vubmvbc.objects import Catalog, Data
import vubmvbc.data_getter as dg

from oma_tracking.oma_clustering import ModeClusterer  #pip install hdbscan --no-build-isolation --no-binary :all
from oma_tracking.data.make_dataset import DatasetGetter
from oma_tracking.data.utils import unpack_mode, get_frequencies, get_rated_data, get_parked_data, read_simulations_csv_files

In [3]:
start  = datetime.datetime(2012,1,1,tzinfo=utc)
stop  = datetime.datetime(2023,7,19,tzinfo=utc)

location = 'bbc01'
name_location = 'BB_C01'

home_folder = "../../"
data_file_name = '_'.join([location, start.strftime("%Y%m%d"), stop.strftime("%Y%m%d")])
data_path = home_folder + "data/bb/raw/" + data_file_name + ".parquet"
data = pd.read_parquet(data_path)


SS1 = get_frequencies(data, 'SS1')
SS2 = get_frequencies(data, 'SS2')
FA1 = get_frequencies(data, 'FA1')

In [4]:
weather_data_path = "../../data/nw2/mvbc_data_longterm.parquet"
Thorntonbank_data_path = "../../data/nw2/Thorntonbank_data_longterm.parquet"
Westhinder_data_path = "../../data/nw2/Westhinder_data_longterm.parquet"

weather_data = pd.read_parquet(weather_data_path)

scada_inputs = data.filter(regex='mean')

inputs = \
    pd.concat(
        [
            weather_data,
            scada_inputs
        ],
        axis=1
    )

selected_columns = \
    [
       'mvbc_WandelaarBuoy_10%_highest_waves',
       'mvbc_WandelaarBuoy_Height_waves_with_period_>_10_s',
       'mvbc_WandelaarMeasuringpile_Flow_direction_cell_3',
       #'mvbc_WandelaarBuoy_Wave_height',
       #'mvbc_WandelaarBuoy_Average_wave_period',
       #'mvbc_WandelaarMeasuringpile_Tide_TAW',
       #'mvbc_WandelaarMeasuringpile_Air_pressure',
       #'mvbc_WandelaarMeasuringpile_Air_temperature',
       #'mvbc_Wandelaar_Sea_water_temperature',
       'Sea_water_temperature',
       'Wave_height',
       'Tide_TAW',
       'Average_wave_period',
       'Air_pressure',
       'Air_temperature',
       'mean_'+ name_location + '_rpm',
       'mean_'+ name_location + '_yaw',
       'mean_'+ name_location + '_pitch',
       'mean_'+ name_location + '_power',
       'mean_'+ name_location + '_windspeed',
       'mean_'+ name_location + '_winddirection'
    ]
#inputs = inputs[selected_columns]

In [8]:
data

,location,mpe_BB_C01_FA1,mpe_BB_C01_SS1,mpe_BB_C01_SS2,mean_BB_C01_rpm,mean_BB_C01_yaw,mean_BB_C01_pitch,mean_BB_C01_windspeed,mean_BB_C01_winddirection,max_BB_C01_TW_ACC_LAT027_FA,...,min_BB_C01_TW_ACC_LAT027_DEG031_Y_nr2,min_BB_C01_TW_ACC_LAT041_DEG031_Y_nr2,min_BB_C01_TW_ACC_LAT069_DEG031_X_nr1,min_BB_C01_TW_ACC_LAT069_DEG031_Y_nr2,rms_BB_C01_TP_ACC_LAT019_DEG315_X_nr1,rms_BB_C01_TP_ACC_LAT019_DEG315_Y_nr2,rms_BB_C01_TW_ACC_LAT027_DEG031_Y_nr2,rms_BB_C01_TW_ACC_LAT041_DEG031_Y_nr2,rms_BB_C01_TW_ACC_LAT069_DEG031_X_nr1,rms_BB_C01_TW_ACC_LAT069_DEG031_Y_nr2
timestamp,,,,,,,,,,,,,,,,,,,,,
2012-01-01 00:00:00+00:00,BBC01,"[{'algorithm': 'ssicov', 'mean_damping': 8.695...","[{'algorithm': 'ssicov', 'mean_damping': 3.859...","[{'algorithm': 'lscf', 'mean_damping': 4.39388...",16.0,241.200000,-0.2,12.1,239.400000,0.055983,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 00:10:00+00:00,BBC01,None,"[{'algorithm': 'lscf', 'mean_damping': 2.75079...",None,16.0,241.500000,1.2,13.0,236.700000,0.050301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 00:20:00+00:00,BBC01,"[{'algorithm': 'lscf', 'mean_damping': 4.52981...","[{'algorithm': 'ssicov', 'mean_damping': 2.592...","[{'algorithm': 'lscf', 'mean_damping': 5.46504...",16.0,238.400000,0.8,13.1,232.900000,0.051119,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 00:30:00+00:00,BBC01,None,"[{'algorithm': 'lscf', 'mean_damping': 3.34041...","[{'algorithm': 'lscf', 'mean_damping': 2.56667...",16.0,235.900000,4.2,14.6,231.700000,0.056162,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 00:40:00+00:00,BBC01,"[{'algorithm': 'lscf', 'mean_damping': 2.82345...","[{'algorithm': 'ssicov', 'mean_damping': 4.521...","[{'algorithm': 'lscf', 'mean_damping': 5.19294...",16.0,234.100000,7.6,16.0,229.400000,0.045163,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-18 23:20:00+00:00,BBC01,"[{'algorithm': 'lscf', 'mean_damping': 9.91676...","[{'algorithm': 'lscf', 'mean_damping': 1.06046...",None,10.0,211.500000,-1.4,4.6,217.199997,0.664631,...,-0.004062,-0.005783,-0.008288,-0.008328,0.001381,0.002537,0.000941,0.001289,0.002193,0.001676
2023-07-18 23:30:00+00:00,BBC01,"[{'algorithm': 'lscf', 'mean_damping': 5.35879...","[{'algorithm': 'lscf', 'mean_damping': 1.56599...","[{'algorithm': 'lscf', 'mean_damping': 2.31766...",10.0,221.399994,-1.5,4.6,219.000000,0.656289,...,-0.004427,-0.006534,-0.016006,-0.008109,0.001085,0.003137,0.001007,0.001652,0.002768,0.001922
2023-07-18 23:40:00+00:00,BBC01,"[{'algorithm': 'lscf', 'mean_damping': 5.79810...","[{'algorithm': 'lscf', 'mean_damping': 2.19270...","[{'algorithm': 'lscf', 'mean_damping': 1.47486...",10.0,218.399994,-1.1,4.4,215.600006,0.667987,...,-0.004154,-0.005952,-0.012271,-0.008001,0.001290,0.003590,0.001009,0.001536,0.002850,0.001713


In [7]:
from oma_tracking.data.make_dataset import get_mpe_data

name_location = 'BB_C01'
mpe_data = {}
directions = ['SS', 'FA']
for direction in directions:
    mpe_data[direction] = get_mpe_data(data, direction, name_location)

KeyError: 'mpe_BB_C01_SS'

In [ ]:
mpe_data

In [ ]:
from oma_tracking.modal_tracking.harmonics import HarmonicDetector
plt.rcParams.update({'font.size': 20})
clustering_data = {}
harmonic_detectors = {}
harmonics_removed = {}
for direction in ['SS']:
    clustering_data[direction] = mpe_data[direction].rename(columns={'mean_frequency':'frequency', 'mean_damping':'damping', 'size':'size'})
    harmonic_detectors[direction] = HarmonicDetector(turbine_data, clustering_data[direction], min_rpm=0, max_distance = 0.08, p_orders=[3,6,9,12, 15])
    distance_calculator_data = harmonic_detectors[direction].get_distance_calculator_data()
    harmonic_detectors[direction].plot_distances()
    harmonic_detectors[direction].plot_harmonics_removed()
    harmonics_removed[direction] = harmonic_detectors[direction].remove_harmonics()
